## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="o3-mini",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [8]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [9]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [10]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 나눌 수 있습니다.\n\n1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.\n\n2. **데이터 전처리**: 수집한 데이터는 모델이 이해할 수 있는 형태로 가공해야 합니다. 예를 들어, 이미지의 크기를 조정하거나, 텍스트를 숫자로 변환하는 등의 작업이 필요합니다.\n\n3. **모델 선택**: 학습할 인공지능 모델을 선택합니다. 예를 들어, 이미지 인식에는 CNN(합성곱 신경망)을, 자연어 처리에는 RNN(순환 신경망)이나 Transformer 모델을 사용할 수 있습니다.\n\n4. **학습**: 모델은 주어진 데이터를 바탕으로 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터와 정답(라벨) 간의 관계를 이해하려고 합니다. 예를 들어, 고양이와 개의 이미지를 학습할 때, 모델은 고양이와 개의 특징을 구분하는 방법을 배우게 됩니다.\n\n5. **손실 함수**: 모델의 예측이 실제 정답과 얼마나 차이가 있는지를 측정하는 손실 함수를 사용합니다. 이 값을 최소화하는 방향으로 모델의 파라미터(가중치)를 조정합니다.\n\n6. **최적화**: 손실 함수를 최소화하기 위해 경량화된 알고리즘(예: 경사 하강법)을 사용하여 모델의 파라미터를 업데이트합니다. 이 과정을 여러 번 반복하면서 모델이 점점 더 정확해집니다.\n\n7. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터(검증 데이터)를 사용합니다. 이 데이터는 모델이 학습할 때 사용하지 않은 데이터로, 모델의 일반화 능력을 확인하는 데 도움을 줍니다.\n\n8. **배포**: 모델이 충분히 학습하고 검증되면, 실제 환경에서 사용할 수 있도록 배포합니다. 이후에도 새로운 데이터로 모델을 업데이트하거나 개선할 수 있습니다.\n\n이러한 과정을 통해 인공지능 모델은 주어진 문제를 해결할 수 있

In [11]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 {how} 설명해주세요.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

chain = prompt | model

# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리", "how": "쉽고 재밌게"}
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)


AIMessage(content='인공지능 모델의 학습 원리를 쉽게 설명해볼게요! 상상해보세요, 여러분이 요리를 배우고 있다고 가정해봅시다. 처음에는 레시피를 보고 재료를 준비하고, 요리를 해보죠. 하지만 처음에는 맛이 별로일 수 있어요. 그럴 때마다 친구나 가족에게 맛을 평가받고, 피드백을 받아서 다음 번에는 더 맛있게 요리할 수 있도록 노력하는 거죠.\n\n이와 비슷하게, 인공지능 모델도 데이터를 통해 학습해요. 여기서 데이터는 요리의 재료와 같아요. 모델은 많은 데이터를 가지고 "요리"를 하면서, 즉 문제를 해결하려고 시도해요. 처음에는 잘 못할 수도 있지만, 정답과 비교하면서 점점 더 나아지죠.\n\n1. **데이터 수집**: 요리 재료를 모으는 것처럼, 모델은 학습할 데이터를 수집해요. 예를 들어, 고양이와 개의 사진을 모은다고 해볼게요.\n\n2. **학습**: 모델은 이 데이터를 가지고 "훈련"을 해요. 고양이와 개의 사진을 보고, 어떤 특징이 있는지 배우는 거죠. 예를 들어, 고양이는 보통 귀가 뾰족하고, 개는 귀가 늘어져 있을 수 있어요.\n\n3. **피드백**: 모델이 예측한 결과가 맞는지 틀리는지를 확인해요. 만약 고양이 사진을 개로 잘못 분류했다면, 그 이유를 분석하고 수정하는 과정을 거쳐요. 이 과정이 반복되면서 모델은 점점 더 정확해져요.\n\n4. **테스트**: 마지막으로, 모델이 잘 학습했는지 확인하기 위해 새로운 데이터를 가지고 테스트해요. 이때 모델이 얼마나 잘 예측하는지 평가하죠.\n\n결국, 인공지능 모델은 많은 데이터를 통해 경험을 쌓고, 피드백을 받아가며 점점 더 똑똑해지는 거예요. 마치 요리사가 여러 번 요리를 해보면서 실력을 키우는 것과 같죠! 이렇게 인공지능 모델은 학습을 통해 문제를 해결하는 능력을 키워나갑니다. 재미있죠?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 500, 'promp

아래는 스트리밍을 출력하는 예시 입니다.

In [12]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리를 쉽게 설명해볼게요! 상상해보세요, 여러분이 요리를 배우고 있다고 가정해봅시다. 처음에는 레시피를 보고 재료를 준비하고, 요리를 시작하죠. 하지만 처음에는 맛이 별로일 수 있어요. 그럴 때마다 친구나 가족에게 맛을 평가받고, 피드백을 받아서 다음 번에는 더 맛있게 요리할 수 있도록 노력하는 거죠.

이와 비슷하게, 인공지능 모델도 데이터를 통해 학습해요. 여기서 데이터는 요리의 재료와 같아요. 모델은 많은 데이터를 가지고 "요리"를 하면서, 즉 문제를 해결하려고 시도해요. 처음에는 잘 못할 수도 있지만, 정답과 비교하면서 점점 더 나아지죠.

1. **데이터 수집**: 요리 재료를 모으는 것처럼, 인공지능은 학습할 데이터를 수집해요. 예를 들어, 고양이와 개의 사진을 모은다고 해볼게요.

2. **학습**: 모델은 이 데이터를 가지고 "훈련"을 받아요. 고양이와 개의 사진을 보고, 어떤 특징이 있는지 배우는 거죠. 예를 들어, 고양이는 귀가 뾰족하고, 개는 귀가 늘어질 수 있다는 것을 학습해요.

3. **피드백**: 모델이 예측한 결과가 맞는지 틀리는지를 확인해요. 만약 고양이 사진을 개로 잘못 분류했다면, 그 오류를 통해 모델은 "아, 이건 고양이구나!"라고 배우게 되죠.

4. **반복**: 이 과정을 여러 번 반복하면서 모델은 점점 더 정확하게 예측할 수 있게 돼요. 마치 요리를 여러 번 해보면서 점점 더 맛있게 만드는 것과 같아요.

5. **테스트**: 마지막으로, 모델이 잘 학습했는지 확인하기 위해 새로운 데이터를 가지고 테스트해요. 이때 모델이 얼마나 잘하는지를 평가하고, 필요하면 다시 학습을 시키기도 해요.

결국, 인공지능 모델은 많은 데이터를 통해 경험을 쌓고, 그 경험을 바탕으로 문제를 해결하는 능력을 키워가는 거예요. 그래서 인공지능이 점점 더 똑똑해지는 거죠! 마치 요리사가 요리를 잘하게 되는 과정과 비슷하답니다.

### 출력파서(Output Parser)


In [14]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [15]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [17]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리", "how": "최대한 짧게"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 주어진 데이터에서 패턴을 인식하고, 이를 바탕으로 예측이나 결정을 내리는 것입니다. 주로 지도학습, 비지도학습, 강화학습의 세 가지 방법을 사용하며, 모델은 손실 함수를 최소화하는 방향으로 파라미터를 조정하여 학습합니다. 이 과정에서 반복적인 피드백을 통해 성능을 개선합니다.'

In [ ]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model
input = {"topic": "인공지능 모델의 학습 원리", "how": "최대한 짧게"}
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 주어진 데이터에서 패턴을 인식하고, 이를 바탕으로 예측이나 결정을 내리는 것입니다. 주로 지도학습, 비지도학습, 강화학습의 세 가지 방법을 사용하며, 모델은 반복적인 과정(훈련)을 통해 오류를 최소화하고 성능을 향상시킵니다. 이 과정에서 손실 함수와 최적화 알고리즘이 중요한 역할을 합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 26, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'stop', 'logprobs': None}, id='run-a5da564f-6072-434d-89e1-e0afee8bfbe7-0', usage_metadata={'input_tokens': 26, 'output_tokens': 100, 'total_tokens': 126, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

: 

In [18]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 주어진 데이터에서 패턴을 인식하고, 이를 기반으로 예측이나 결정을 내리는 것입니다. 주로 지도 학습, 비지도 학습, 강화 학습의 세 가지 방법을 사용하며, 모델은 반복적인 과정인 학습을 통해 가중치를 조정하여 성능을 향상시킵니다. 데이터와 알고리즘의 조합을 통해 최적의 결과를 도출하는 것이 핵심입니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [21]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [22]:
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\n당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.\n양식은 [FORMAT]을 참고하여 작성해 주세요.\n\n#상황:\n{question}\n\n#FORMAT:\n- 영어 회화:\n- 한글 해석:\n')

In [23]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [24]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  A: Hi there! Welcome to our restaurant. How many people are you dining with today?  
  B: Just one, please.  
  A: Great! Here’s the menu. Can I get you something to drink?  
  B: Yes, I would like a glass of water, please.  
  A: Sure! Are you ready to order your meal?  
  B: Yes, I’d like the grilled chicken with a side salad, please.  
  A: Excellent choice! Would you like any dressing with your salad?  
  B: Yes, a balsamic vinaigrette, please.  
  A: Perfect! I’ll have that right out for you.  

- 한글 해석:
  A: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 몇 분이서 식사하시나요?  
  B: 저 혼자입니다.  
  A: 좋습니다! 여기 메뉴입니다. 음료수는 드릴까요?  
  B: 네, 물 한 잔 주세요.  
  A: 알겠습니다! 식사 주문할 준비가 되셨나요?  
  B: 네, 그릴에 구운 치킨과 사이드 샐러드를 주문할게요.  
  A: 훌륭한 선택입니다! 샐러드에 드레싱은 어떤 걸 원하시나요?  
  B: 네, 발사믹 비네그레트를 주세요.  
  A: 완벽합니다! 곧 가져다 드리겠습니다.


In [25]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
Customer: Hi, I’d like to see the menu, please.  
Waiter: Of course! Here you go. Are you ready to order?  
Customer: Yes, I’d like to have the grilled chicken with a side salad.  
Waiter: Great choice! Would you like anything to drink?  
Customer: Yes, I’ll have a glass of water, please.  
Waiter: Perfect! I’ll get that for you right away.  

- 한글 해석:  
고객: 안녕하세요, 메뉴를 보고 싶어요.  
웨이터: 물론이죠! 여기 있습니다. 주문할 준비 되셨나요?  
고객: 네, 구운 치킨과 사이드 샐러드를 먹고 싶어요.  
웨이터: 좋은 선택입니다! 음료수는 무엇을 드릴까요?  
고객: 네, 물 한 잔 주세요.  
웨이터: 완벽합니다! 바로 가져다 드릴게요.

In [26]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
Customer: Hi, I'd like to order a pizza, please.  
Pizza Place: Sure! What size would you like?  
Customer: I’ll have a large, please.  
Pizza Place: Great! What toppings do you want?  
Customer: Can I get pepperoni and mushrooms?  
Pizza Place: Absolutely! Would you like any sides or drinks with that?  
Customer: Yes, I’d like a garlic bread and a 2-liter soda, please.  
Pizza Place: Perfect! Your total comes to $25. Would you like to pick it up or have it delivered?  
Customer: I’ll pick it up. How long will it take?  
Pizza Place: It should be ready in about 20 minutes.  
Customer: Thank you!  

- 한글 해석:  
고객: 안녕하세요, 피자 하나 주문할게요.  
피자 가게: 물론이죠! 어떤 사이즈로 하시겠어요?  
고객: 큰 사이즈로 해주세요.  
피자 가게: 좋습니다! 어떤 토핑을 원하시나요?  
고객: 페퍼로니와 버섯으로 해도 될까요?  
피자 가게: 물론이죠! 사이드나 음료는 필요하신가요?  
고객: 네, 마늘빵과 2리터 음료수 하나 주세요.  
피자 가게: 완벽해요! 총 금액은 25달러입니다. 픽업하시겠어요, 아니면 배달하시겠어요?  
고객: 픽업할게요. 얼마나 걸릴까요?  
피자 가게: 약 20분 후에 준비될 거예요.  
고객: 감사합니다!